In [ ]:
#This code is used to determine the mutational robustness of a given network

#It has two modes, relative and absolute scoring

#We take a network and determine it's expected score value by averaging it out for a few runs
#Then we make P networks all with a single mutation in it and then determine their average score

#Once we have this we can calculate the absolute and relative score of these mutated networks

#We will have the option to toggle perturbed initial conditions, asymmetrical splitting and ODE noise all seperately

#We would hope to see that as simulations go on, that their mutational robustness increases

In [ ]:
using Random
using Plots
using Distributions
using DifferentialEquations
using Statistics
using ProgressMeter
using TickTock
using LinearAlgebra
using StatsBase
using LaTeXStrings
using MultivariateStats
using Dates
using Distances
using JLD
using KernelDensity
using Plots
using StatsPlots




function Setting_Up_Randomly(M,Sparse,rng=MersenneTwister(123))
    "Function used to randomly with seed define our c_i's, J_ij's and the initial positions
    Input:
    M      = Amount of Genes in our system
    Sparse = Bool to determine if we have a sparse graph
    rng    = Random Julia seed to make sure we can recreate the same initial conditions
    Output:
    CArray          = C_i's used, can only be between -0.5 and 0.5 now
    PositionArray   = Random Initial position, between -1 and 1 for all genes, epifactors = 0
    ParameterMatrix = J_ij's Currently a dense network with values between -0.5 and 0.5
    "
    ParameterMatrix = zeros(M,M)
    CArray = zeros(1,M)
    PositionArray = zeros(2,1,M)
    
    CArray = rand(rng,Float32,M).-0.5; #Random c_j between -0,5 and 0.5
    PositionArray[1,1,1:M] = 2*(rand(rng,Float32,M).-0.5); #Random Initial position 
    ParameterMatrix = 2 * bitrand(rng,M,M).-1; #J_jk's
    if Sparse
       for i=1:M
            for j=1:M
                if rand(rng) > 5/M #Average of 5 outgoing connections per gene

                    ParameterMatrix[i,j] = 0 
                end
            end
        end
    end
    return CArray, PositionArray, ParameterMatrix
end







function Noisy_Splitting(solution,Total_Splits,M,Seed)
    " Once our cell decides to split, we decide to introduce some random noise in how these genes get distributed.
    This function takes the cells, doubles them and then gives the daughters cell the same gene parameters +- noise
    one daughter gets + the other gets -. For the epigeneitc factors they are simply inherited and do not see any noise
    Input:
    solution     = These are the initial conditions of the parent cells that we need to split 
    (First genes of all cells, then all epigenetic factors)
    Total_Splits = Amount of times the cell has split (including this current split)
    M            = Amount of Genes
    Seed         = Random Noise seed for reproducing results
    Output:
    SplittedArray = New array of the genes and epigenetic factors of the daughter cells
    How is this structured? Suppose we take M=3 for convenience, we would have [g1,g2,g3,e1,e2,e3] for the first parent cell
    The daughters become: [g11,g12,g13,g21,g22,g23,e11,e12,e13,e21,e22,e23]
    Where we have genes of daughter 1, then genes of daughter 2, then epi of daughter 1 and epi of daughter 2.
    When the Daughters split we get the genes of Children of cell 1, then genes of Children of cell 2 and same for epi.
    "
    Random.seed!(Seed)
    s = Total_Splits #For shortening
    
    SplittedArray = zeros(Float32,2,2^(s),M) #2^(s+1), 2M is the normal length, 4M after 1 split, 8M after 2 splits etc.
    for g=1:2^(s-1) #looping over the length of the parent
        for t=1:M
            Gaussian = Normal{Float32}(0.0f0,Splitting_Strength)
            Displace = rand(Gaussian) #Noise for split
            #index = Int32.(floor((g-1)/M)*M +g) #example M=10: Index 1-> 1 and 11, index 2->2 and 12 ... 
            #index 10->10 and 20, index 11->21 and 30 and so on. This is old method

            #Genes
            SplittedArray[1,2*g-1,t] = solution[1,g,t] + Displace
            SplittedArray[1,2*g,t] = solution[1,g,t] - Displace
        end
        #Epigenetic
        SplittedArray[2,2*g-1,:] = solution[2, g,:]
        SplittedArray[2,2*g,:] = solution[2, g,:]   
    end
    return SplittedArray  
end

function Differentiaton_Detection_Parameter(M,Epigenetic_Plot)
    "Function used to convert epigenetic factors into cell types and bits
    Input:
    M               = Amount of Genes 
    Epigenetic_Plot = Array of epigenetic factors which have to be converted to 0 and 1 representing on or off
    Output:
    b        = Array of scalar values representing cell types
    BitGenes = Array of vectors representing the genes of the cell types
    "
    b = Int128[]
    Counter = Int[]
    Epi_Bitwise = signbit.(-Epigenetic_Plot)

    
    BitGenes = [] #We will put the bit notation of the gene in here
    #Later rewrite to let this replace Bit_Represent
    
    for i=1:length(Epi_Bitwise[:,1])
        Bit_Represent::Int128 = 0 #CAREFUL WITH M>127
        for j=1:M
            Bit_Represent += Int128(Epi_Bitwise[i,j]*2)^(j-1)
        end
        #print(Bit_Represent)
        #print("     ")
        if !(Bit_Represent in b)

            push!(BitGenes,collect(Epi_Bitwise[i,:]))
            push!(Counter,1)
        end
        push!(b,Bit_Represent)

        
    end
    
    #print(b)
    Counter = sort(Counter,rev=true)
    #print(Counter)
    Score = 0
    #Make here the formula to turn b and the counter into a score


    return b,BitGenes
end

function Entropy_Score(M,Epigenetic_Plot)
    "Function used to convert epigenetic factors into cell types and bits to then calculate the entropic score.
    See previous function for alternative, this is currently not in use.
    Input:
    M               = Amount of Genes 
    Epigenetic_Plot = Array of epigenetic factors which have to be converted to 0 and 1 representing on or off
    Output:
    Score = Entropy score of the cell types
    "
    b = Int[]
    Counter = Int[]
    Epi_Bitwise = signbit.(-Epigenetic_Plot)
    
    for i=1:length(Epi_Bitwise[:,1])
        Bit_Represent::Int128 = 0 #CAREFUL WITH M>127
        for j=1:M
            Bit_Represent += Int128(Epi_Bitwise[i,j]*2)^(j-1)
        end
        #print(Bit_Represent)
        #print("     ")
        if !(Bit_Represent in b)
            push!(b,Bit_Represent)
            push!(Counter,1)
        else 
            Counter[findfirst(==(Bit_Represent), b)] += 1
        end
        
    end
    
    #print(b)
    Counter = sort(Counter,rev=true)
    #print(Counter)
    Score = 0
    #Make here the formula to turn b and the counter into a score
    for i=1:length(b)
       prob = Counter[i]/M
       Score -= prob*log(prob)
    end
    
    return Score
end



function Mutate_Parameters(ParameterMatrix_Mut,M,Mutations,Seed)
    "Function used to mutate the network (we turn a 1 or -1 to 0 and we turn a 0 to 1 or -1)
    Input:
    ParameterMatrix_Mut = Original Network that we want to mutate
    M                   = Amount of Genes 
    Mutations           = How many connections will we mutate
    Seed                = Randomness seed
    Output:
    ParameterMatrix_Mut = Mutated Network
    "
    Zero_Indices = findall(iszero,ParameterMatrix_Mut)
    One_Indices = findall(!iszero,ParameterMatrix_Mut)
    Random.seed!(Seed)
   
    for j=1:Mutations 
        t_zero = rand(1:length(Zero_Indices))
        t_one = rand(1:length(One_Indices))
        i_zero = Zero_Indices[t_zero][1]
        j_zero = Zero_Indices[t_zero][2]
        i_Value = One_Indices[t_one][1]
        j_Value = One_Indices[t_one][2]



        ParameterMatrix_Mut[i_zero,j_zero] = rand(0:1)*2-1 #turn a zero into 1 or -1
        ParameterMatrix_Mut[i_Value,j_Value] = 0 #turn 1 or -1 into 0




        
    end
    return ParameterMatrix_Mut
end


#Some Julia functions I played around with, may reconsider using more later
MultiplyBroad_(a,b) = a .* b

SummingDim2_(a) = sum(a,dims=2) 

HyperTanh_(a) = tanh.(40*a)



function Epigenetic_Evolution3(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:2^Splits

        du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    
        du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] 
            #Epigenetic Evolution, no need to worry about boundaries here so simple loop      
    end  
    
end



function Gradient(u,Splits,t)
    "Function used to calculate the gradient of a cells neighbours, takes into accounts if there even are neighbours.
    Input:
    u      = Gene concentrations over which we take the gradient
    Splits = How many times have the cells split already (determines where neighbours are)
    t      = What location is our cell (boundaries only have 1 neighbour)
    Output:
    Multiple different version depending on the splits, this will be the gradient we want
    "
    if Splits == 0
        return 0
    elseif Splits ==1
        if t==1
            return u[t+1,:] - u[t,:]
        else
            return u[t-1,:] - u[t,:]
        end
    else
        if t==1
            return u[t+1,:] - u[t,:]
        elseif t==2^Splits
            return u[t-1,:] - u[t,:]
        else
            return u[t-1,:] - 2*u[t,:] + u[t+1,:]
        end
    end
    
end

function Epigenetic_Evolution4(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:2^Splits

        du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    
        du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] +KArray.*Gradient(u[1,:,:],Splits,t)
            #Epigenetic Evolution, no need to worry about boundaries here so simple loop      
    end  
    
end

function σ_Genes(du, u,Parameters,Time)
    du .= Noise
    
end



function Epigenetic_Evolution2(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    Cells::Int32 = 2^Splits
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:Cells  

       du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    

       du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] .+ KArray.*(vec(sum(u[1,:,:],dims=1))/Cells - u[1,t,:])

    end  
    
end

#Defining it all in function form


function Mutating_Network(N,M,K,ParameterMatrix_Collection,PositionArray_Collection, Indices, Counter, Genetics)
    "Function used to make mutated copies of the best performing networks. Currently not in use
    Input:
    N                          = How many copies of the network do we make
    M                          = Amount of Genes
    K                          = Supposed to work as a fraction of N, telling us how many mutant versions
    ParameterMatrix_Collection = The original networks
    PositionArray_Collection   = Original Positions
    Indices                    = Sorted indices giving the best networks
    Counter                    = Randomiser seed
    Genetics                   = Used to track heritance of networks
    Output:
    ParameterMatrix_Collection = Updated network
    PositionArray_Collection   = Updated initial positions
    Counter                    = Updated seed
    Updated_Genes              = Updated inheritance
    "
    Updated_Genes = zeros(N)
    Mutations = 1
    Seed_Mutations = 1234
    Indexation_Mutate = 1
    Temporary_Matrix = zeros(Float32,N,M,M)
    Temporary_Vector = zeros(Float32,N,2,1,M)
    for p = 1:K
        for q=1:N/(2*K)
            Temporary_Matrix[Indexation_Mutate,:,:] = Mutate_Parameters(ParameterMatrix_Collection[Indices[p],:,:],M,Mutations,Seed_Mutations+Counter)
            Temporary_Vector[Indexation_Mutate,:,:,:] = PositionArray_Collection[Indices[p],:,:,:]
            Updated_Genes[Indexation_Mutate] = Genetics[p]
            Temporary_Matrix[Indexation_Mutate+1,:,:] = Temporary_Matrix[Indexation_Mutate,:,:]
            Temporary_Vector[Indexation_Mutate+1,:,:,:] = Temporary_Vector[Indexation_Mutate,:,:,:]
            Updated_Genes[Indexation_Mutate+1] = Genetics[p]
            Counter += 1
            Indexation_Mutate +=2
        end
        Temporary_Matrix[Indexation_Mutate-1,:,:] = ParameterMatrix_Collection[Indices[p],:,:] #We overwrite it to add the original back
        Temporary_Matrix[Indexation_Mutate-2,:,:] = ParameterMatrix_Collection[Indices[p],:,:]
    end
    ParameterMatrix_Collection = deepcopy(Temporary_Matrix);
    PositionArray_Collection = deepcopy(Temporary_Vector);
    return ParameterMatrix_Collection, PositionArray_Collection, Counter, Updated_Genes
end



function Running_Network(CArray,ParameterMatrix,PositionArray,Random_Noise)
    "Function used to run our simulation
    Input:
    CArray          = Constants used in our ODE solver  
    ParameterMatrix = Collection of Networks
    PositionArray   = Original Positions
    Random_Noise    = Randomiser seed
    Output:
    PositionArray[2,:,:] = Epigenetic factors from our simulation
    Random_Noise         = Updated seed
    "
    #CArray = CArray_Collection[x,:]
    #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
    #PositionArray = PositionArray_Collection[x,:]
    Orbit_Collection = zeros(Float32,2^Splits*M+1,0)

    
    
    
    
    for s=1:Splits #Looping over the splits
        Parameters = (ParameterMatrix, CArray,KArray,Int32(s-1)) #We have to update the splits every rerun, rest stays same
        tspan = (Float32(0.0),SplitTime) #How long ODE will run
        if Mean_Field
            prob = ODEProblem(Epigenetic_Evolution2,PositionArray,tspan,Parameters) #ODEdefining
        elseif Diffusing_System
            prob = ODEProblem(Epigenetic_Evolution4,PositionArray,tspan,Parameters) #ODEdefining
        elseif Stochastic_Noise
            prob = SDEProblem(Epigenetic_Evolution4, σ_Genes,PositionArray,tspan,Parameters)

        else
            prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
        end    #Solving
        if !Stochastic_Noise
            sol = solve(prob,BS3())
        else
            sol = solve(prob,RKMilCommute())
            #sol = solve(prob,LambaEulerHeun())
        end
        Times = sol.t #Save the time, doesn't get used yet though
        StepAmount = length(Times)  #Amount of data points per gene
        Genes = sol 
        PositionArray = Genes[StepAmount] #Saving the genes and epi factors
        
        OrbitalCollection = Times' .+ (s-1)*SplitTime
        for i=1:2^(Splits-s+1)  #s=1 this gives all 2^Splits, this is to say how much we repeat same path
            for j=1:2^(s-1) #Together multiply, always 2^Splits orbits, these j represent different paths
                OrbitalCollection = vcat(OrbitalCollection,sol[1,j,:,:])                
            end
        end
        Orbit_Collection = hcat(Orbit_Collection,OrbitalCollection)
        
        #Orbit1 = [Times,Genes[1,1,1:M,1:StepAmount]]
        #Orbit2 = [Times,Genes[1,1,1:M,1:StepAmount]]
        #EpiOrbit1 = [Times,Genes[2,1,1:M,1:StepAmount]]
        #EpiOrbit2 = [Times,Genes[2,1,1:M,1:StepAmount]]
        #Now we must take all the values from the updated PositionArray and give appropriate terms to the daughter cells
        PositionArray = Noisy_Splitting(PositionArray,s,M,Random_Noise)  #Splitting function
        Random_Noise += 1
    end

    Final_Run_Time::Int32 = 20000 #Long evolving of the cells after splitting to get the final epigenetic state
    Parameters = (ParameterMatrix, CArray,KArray,Int32(Splits)) 
    tspan = (Int32(0.0),Final_Run_Time)

    if Mean_Field
        prob = ODEProblem(Epigenetic_Evolution2,PositionArray,tspan,Parameters) #ODEdefining
    elseif Diffusing_System
        prob = ODEProblem(Epigenetic_Evolution4,PositionArray,tspan,Parameters) #ODEdefining
    elseif Stochastic_Noise
        prob = SDEProblem(Epigenetic_Evolution4, σ_Genes,PositionArray,tspan,Parameters)

    else
        prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
    end    #Solving
    if !Stochastic_Noise
        sol = solve(prob,BS3())
    else
        sol = solve(prob,RKMilCommute())
    end
    #sol = solve(prob,alg_hints = [:nonstiff]) #Solving
    #sol = solve(prob)
    Times = sol.t
    StepAmount = length(Times)
    GenesEpi = sol
    PositionArray = GenesEpi[StepAmount]

    OrbitalCollection = Times' .+ Splits*SplitTime 
    for j=1:2^(Splits) #Together multiply, always 2^Splits orbits, these j represent different paths
        OrbitalCollection = vcat(OrbitalCollection,sol[1,j,:,:])                
    end
    Orbit_Collection = hcat(Orbit_Collection,OrbitalCollection)
    
    #Orbit1 = [cat(Orbit1[1],Times.+SplitTime,dims=1),cat(Orbit1[2],GenesEpi[1,1,1:M,1:StepAmount],dims=2)]
    #Orbit2 = [cat(Orbit2[1],Times.+SplitTime,dims=1),cat(Orbit2[2],GenesEpi[1,2,1:M,1:StepAmount],dims=2)]
    #EpiOrbit1 = [cat(EpiOrbit1[1],Times.+SplitTime,dims=1),cat(EpiOrbit1[2],GenesEpi[2,1,1:M,1:StepAmount],dims=2)]
    #EpiOrbit2 = [cat(EpiOrbit2[1],Times.+SplitTime,dims=1),cat(EpiOrbit2[2],GenesEpi[2,2,1:M,1:StepAmount],dims=2)]

    return PositionArray[2,:,:],Random_Noise, Orbit_Collection
end


function Scoring_Indexes(N,Final_Epigenetic_Factors,Cell_Bit)
    "Function used to calculate the score of all the networks, distribution robustness and the cell type robustness. Note
    that we are still selecting on amount of different cells, not entropy.
    Input:
    N                       = How many copies of the same networks we have 
    Final_Epigentic_Factors = Array of epigenetic factors after running the simulations
    Cell_Bit                = Cell type tracker
    Output:
    Indices                   = Indices of networks sorted by score, we only select these
    AllIndices                = Longer copy of Indices, contains also those we will no longer use
    Evolution_Indices         = Gives us the score corresponding with the Indices we selected on
    [Cell_Types,Cell_Amounts] = Vector of the cell types and how many of these cell types there are
    Cell_Bit                  = Updated cell type tracker
    KLD_Distance              = Kullback-Lieber divergence score for the distribution robustness
    Bit_Distances             = Cell Type robustness measure
    "
    Scoring = zeros(2,N)
    Cell_Types = []
    Cell_Amounts = []
    Cell_Distribution = Array{Vector{Float64}}(undef, N) 
    Cell_Bit_Array = Array{Vector{Any}}(undef, N)

    for x=1:N
        Differentiation, Bit_Genes = Differentiaton_Detection_Parameter(M,Final_Epigenetic_Factors[x,:,:]) 
     
        #Here we save some cell types and bits to use for our robustness calculation after the for loop
        #Cell types
        alpha = countmap(Differentiation)
        Temp_Array = Vector{Float64}()
        for i in keys(alpha)
            Temp_Array = push!(Temp_Array,alpha[i])
        end
        #print(Temp_Array)
        Cell_Distribution[x] = sort(Temp_Array,rev=true)/2^Splits #We sort the distribution of our cells and normalise
        #Bits
        Cell_Bit_Array[x] = Bit_Genes
        
        
        
        
        #This part here is where we track the cell types we had before as well
        for tau in Bit_Genes
            if !(tau in Cell_Bit)

                push!(Cell_Bit,tau)


            end
        end
        for tau in Differentiation 
            if !(tau in Cell_Types)
                push!(Cell_Types,tau)
                push!(Cell_Amounts,1)
            else
                Cell_Amounts[findfirst(==(tau), Cell_Types)] += 1
            end
        end

        Scoring[1:2,x] = [length(countmap(Differentiation)),x]

    end

    #Robustness calculation
    KLD_Distance = 0
    
    KLD_Adder = 0
    for i=1:N
        for j=1:N #Loop over all and calculate KL for all distributions
            if i!=j #If the same we get no difference anyways
                Length_Differ = length(Cell_Distribution[i])-length(Cell_Distribution[j])
                if Length_Differ > 0 #i is longer so we need to add 0 to j
                    KLD_Adder = kldivergence(Cell_Distribution[i],vcat(Cell_Distribution[j],zeros(Length_Differ)))
                    if KLD_Adder != Inf #There are zeros in the second distribution (divide by 0 error)
                        KLD_Distance += kldivergence(Cell_Distribution[i],vcat(Cell_Distribution[j],zeros(Length_Differ)))
                    end
                else
                    KLD_Adder = kldivergence(vcat(Cell_Distribution[i],zeros(-Length_Differ)),Cell_Distribution[j])

                    KLD_Distance += kldivergence(vcat(Cell_Distribution[i],zeros(-Length_Differ)),Cell_Distribution[j])
                end
            end
        end
    end
    KLD_Distance = KLD_Distance/(N*(N-1))
    
    #Now we see the closest type in the other groups
    Bit_Distances = 0
    for x =1:N
        for Bit_Inside in Cell_Bit_Array[x] #These are all the cell types in bit form
            for y =1:N #We loop over the others
                if x!=y #Must look at different otherwise distance 0 again
                    Bit_Distan_Tem = M
                    for Bit_Outside in Cell_Bit_Array[y] #We loop over all the cell types in this other one
                        Bit_Distan_Temp = sum(abs.(Bit_Inside .- Bit_Outside)) 
                        if Bit_Distan_Temp < Bit_Distan_Tem #Distance must be smaller if it is the best fit
                            Bit_Distan_Tem = Bit_Distan_Temp
                        end
                    end
                    Bit_Distances += sqrt(Bit_Distan_Tem)/(N*(N-1)) #Sqrt to make euclidean distance, N*(N-1) for normalisation
                end
            end
        end
    end
    
    
    #Here we mess a bit with the indices to just have the ones with the highest score later
    
    Indices = sortslices(Scoring,dims=2,rev=true)
    Evolution_Indices = Indices[1,1:N]

    AllIndices = deepcopy(Indices[2,:])
    Indices = Indices[2,1:N]
    Indices = Int.(Indices)
    #print(Indices[2,1:K])
    return Indices, AllIndices, Evolution_Indices, [Cell_Types,Cell_Amounts],Cell_Bit, KLD_Distance, Bit_Distances
end

function Epi_Plotter(Indices,K,Final_Epigenetic_Factors)
    "Function used to plot the epigenetic factors of different cells
    Input:
    Indices                 = Indices of highest scoring runs  
    k                       = 4 or 8 plots
    Final_Epigentic_Factors = Epigenetic factors that we will plot
    Output:
    k = Plot element, simply use display on the output to see the requested plot
    "
    pa = plot(Final_Epigenetic_Factors[Indices[1],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pb = plot(Final_Epigenetic_Factors[Indices[2],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pc = plot(Final_Epigenetic_Factors[Indices[3],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pd = plot(Final_Epigenetic_Factors[Indices[4],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    if K==8
        pe = plot(Final_Epigenetic_Factors[Indices[5],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        pf = plot(Final_Epigenetic_Factors[Indices[6],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        pg = plot(Final_Epigenetic_Factors[Indices[7],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        ph = plot(Final_Epigenetic_Factors[Indices[8],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        k = plot(pa,pb,pc,pd,pe,pf,pg,ph,layout=(4,2),legend=false)
    else
        k = plot(pa,pb,pc,pd,layout=(2,2),legend=false)
    end
    return k
end

function Gene_Variance(Final_Epigenetic_Factors,M,N)
    "Function used to calcualte the variance in expressed genes
    Input:
    Final_epigenetic_Factors = Final epigenetic values which represent gene expression
    M                        = Amount of Genes
    N                        = How many copies of same network do we have
    Output:
    Deviation = Variance in these genes (normalised by 2^Splits*N)
    "
    BitterMatrix = signbit.(-Final_Epigenetic_Factors[:,:,:])
    Average_Bits = zeros(M)
    for i=1:N
        for j=1:2^Splits
            Average_Bits += BitterMatrix[i,j,:]
        end
    end
    Average_Bits /= N*2^Splits
    Deviation = 0

    for i=1:N
        for j=1:2^Splits
            Deviation += sum((Average_Bits .- BitterMatrix[i,j,:]).^2)/(2^Splits*N-1)
        end
    end
    
    return Deviation
end

function PCA_Distribution_Creator(PCA_Dimensions,Transformed,Dimensions_Distribution,Cell_Distribution)
    
    
    PCA_Dimensional_Values = zeros(PCA_Dimensions,findmax(Dimensions_Distribution)[1]) 
    #Here we will save all the values and use these for indexes later
    B = Array{Vector{Tuple{Float64, Int64}}}(undef, PCA_Dimensions)
    for j=1:PCA_Dimensions
        B[j] = [(i, count(==(i), Transformed)) for i in unique(Transformed[j,:])] #Here we find what the arguments are for the
        #different dimensions
        for k = 1:length(B[j])
            PCA_Dimensional_Values[j,k] = B[j][k][1] #Here we save all the individual arguments in the corresponding dimension
        end
    end
    #print(PCA_Dimensional_Values)   #If we wanna see them

    Components_Length = length(Transformed[1,:]) 
    for k in 1:Components_Length
        Index_PCA = zeros(PCA_Dimensions)

        for l=1:PCA_Dimensions
            Index_PCA[l] = (findfirst(==(Transformed[l,k]), PCA_Dimensional_Values[l,:]))
        end 
        Index_PCA = Int.(Index_PCA)
        
        
        if PCA_Dimensions==2 #I really tried but I couldn't fix it so just throwing in this hack
            Cell_Distribution[Index_PCA[1],Index_PCA[2]] += 1/Components_Length
        elseif PCA_Dimensions==1
            Cell_Distribution[Index_PCA[1]] += 1/Components_Length   
        elseif PCA_Dimensions==3
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3]] += 1/Components_Length
        elseif PCA_Dimensions==4
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3],Index_PCA[4]] += 1/Components_Length
        elseif PCA_Dimensions==5
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3],Index_PCA[4],Index_PCA[5]] += 1/Components_Length
        else
        print("ADD MORE HERE THERE IS A MISTAKE, DO NOT GO OVER 5 PCA DIMENSIONS YET")
        end
    end
    

    
    return Cell_Distribution
end





In [ ]:
#Write the general initialisation here
Data_To_Analyse = "High_ODE_No_Split"


Copies = [1] #ONLY MAKE THIS A SINGULAR ELEMENT
Load_Generations = [30*i for i=1:25]
#Main variables to change:
N_Initial::Int32 = 64 #How many copies of same network we have
N_Mutated::Int32 = 32 #How many copies of same network we have
P = 32 #How many mutated networks will we make


#DO NOT CHANGE THESE
Splits::Int32 = 3
SplitTime::Float32 = 25 #Time before a splitting event happens, DOES NOT MATTER HERE
M::Int32 = 100 #This is the amount of genes we want in our network
#CAREFUL WITH M>127 DO NOT INCREASE WITHOUT REWRITING DIFFERENTIATION FUNCTION
#BIT OVERFLOW!!!!!


#Constant and plotting tools
a::Int32 = 1.0   #Ratio of gene to epigenetic factor
v::Float32 = 6*10^-4  #Speed at which epigenetic factor changes

InvSqr::Float32 = 1/sqrt(M)
PCA_Plotting = false 
Orbit_Plotting = true  #Currently not used yet
Epi_Plotting = false #Currently not used yet
Counter = 0 #Used to randomise the mutations, leave to keep same seed


#Diffusion or mean field parameters
Mean_Field = false
Diffusing_System = false
KArray = zeros(Float32,M,1) #Diffusion/Mean Field array is set to zero everywhere
#KArray[1:10] .= 0.1 #Here we can manually adjust which genes will have diffusion
Stochastic_Noise = true
Noise = 0.001
Splitting_Strength = 0.001f0




#Amount_Large_Perturbations = 1 #How many large perturbations we will have each time
Amount_Small_Perturbations = N_Mutated #How many smaller perturbations we will have around our large perturbation
#Large_Pert_Radius = 0
Small_Pert_Radius = 0
Perturbations = zeros(64)


Generation_String = string(Load_Generations[1])
Copy_String = string(Copies[1])
Orbit_Array = 0
Mut_Seed = 1237




#Also add what dataset we will take and analyse
#Add how many and which generations of it we will take


In [ ]:
Generation_String = string(Load_Generations[1])
Copy_String = string(Copies[1])
Load_Data_String = "Data_All/"*Data_To_Analyse
Orbit_Array = 0
Counter_EMD_Index = 0

#Write a for loop that goes over all different generations

#Then start analysing our initial network and get it's score value

#Then we write a for loop over P that generates a mutated network and calculates an average score value for it
#We save this score and use a function to determine if it fullfills our condition (did the mutation retain original score enough?

#EXTRA EXTRA
#Expand this code by also calculating if the distribution or cell types have remained?

#End the P loop here
#Make an updated plot of the mutational robustness

#End the generation loop

In [ ]:
Initial_Mean = zeros(length(Load_Generations))
Initial_Std = zeros(length(Load_Generations))
Mutated_Mean = zeros(length(Load_Generations))
Mutated_Std = zeros(length(Load_Generations))
@showprogress 1 for zeta in Load_Generations
    Counter_EMD_Index += 1
    
    for iota in Copies #DO NOT USE MORE COPIES YET
        Generation_String = string(zeta)

        Copy_String = string(iota)
        Current_Network_String = "/Gen:"*Generation_String*"_"*Copy_String*".jld"
        Loaded_Data = load(Load_Data_String*Current_Network_String)
        mkpath(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Mutational_Robustness")



        #Here we load and perturb the networks
        Perturb_Seed = 12345
        rng=MersenneTwister(Perturb_Seed)

        CArray_Collection = zeros(Float32,M)
        PositionArray_Collection = zeros(Float32,P,Amount_Small_Perturbations,2,1,M)
        ParameterMatrix_Collection = zeros(Int32,P,M,M)
        CArray_Collection = Loaded_Data["CArray"]
        for p=1:P
            ParameterMatrix_Collection[p,:,:] = Loaded_Data["Network"]
            #Write Network mutation!
            ParameterMatrix_Collection[p,:,:] = Mutate_Parameters(ParameterMatrix_Collection[p,:,:],M,1,Mut_Seed)
            Mut_Seed += 1
            for i=1:Amount_Small_Perturbations


                #Small pert vector here
                Noise_Vector2 = randn(rng,M).-0.5
                Norm = sqrt(sum(Noise_Vector2.^2))
                Noise_Vector2 = rand()^(1/M)*Small_Pert_Radius*Noise_Vector2/Norm

                PositionArray_Collection[p,i,1,1,:] = Loaded_Data["PositionArray"] + Noise_Vector2'

            end
        end

        


        Random_Noise = 574 #seed for splitting noise

        #Need to define outside for loop, otherwise it will be destroyed afterwards
        Genes = 0





        KLDistance_temp = 0
        BitDistance_temp =  0
        Indices = 0
        Information = zeros(1)
        Ratio = zeros(1)
        Cell_Type_Tracker = []
        Cell_Collection = []
        Temporary = []
        Time_Orbital_Collection = 0

        
        #Write the initial network run here as well!
        
        PositionArray_Initial = zeros(Float32,N_Initial,2,1,M)
        ParameterMatrix_Initial = zeros(Int32,M,M)
        for i=1:N_Initial
            ParameterMatrix_Initial[:,:] = Loaded_Data["Network"]
            
            Noise_Vector2 = randn(rng,M).-0.5
            Norm = sqrt(sum(Noise_Vector2.^2))
            Noise_Vector2 = rand()^(1/M)*Small_Pert_Radius*Noise_Vector2/Norm
            PositionArray_Initial[i,1,1,:] = Loaded_Data["PositionArray"] + Noise_Vector2'
        end
        
        Final_Initial_Epi_Factors = zeros(Float32,N_Initial,2^(Splits),M)
        for i=1:N_Initial #Loop over unperturbed network for expected score value 
            Final_Initial_Epi_Factors[i,:,:],Random_Noise,Temporary = Running_Network(CArray_Collection,ParameterMatrix_Initial,PositionArray_Initial[i,:,:,:],Random_Noise)
        end
        Cell_Type_Tracker = []
        Initial_Evolutionary_Complexity = zeros(N_Initial)
        Indices,AllIndices,Initial_Evolutionary_Complexity[:],Cell_Collection, Cell_Type_Tracker,KLDistance_temp,BitDistance_temp = Scoring_Indexes(N_Initial,Final_Initial_Epi_Factors,Cell_Type_Tracker)

        Initial_Mean[Counter_EMD_Index] = mean(Initial_Evolutionary_Complexity/2^Splits)
        Initial_Std[Counter_EMD_Index] = std(Initial_Evolutionary_Complexity/2^Splits)
        Mutated_Evolutionary_Complexity = zeros(P,Amount_Small_Perturbations)
        Final_Epigenetic_Factors = zeros(Float32,P,Amount_Small_Perturbations,2^(Splits),M)
        for p=1:P #Loop over different networks
            for x=1:Amount_Small_Perturbations #Loop over copies of same network (can be different initial)
                #CArray = CArray_Collection[x,:]
                #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
                #PositionArray = PositionArray_Collection[x,:]
                Final_Epigenetic_Factors[p,x,:,:],Random_Noise,Temporary = Running_Network(CArray_Collection,ParameterMatrix_Collection[p,:,:],PositionArray_Collection[p,x,:,:,:],Random_Noise)

            end
            Cell_Type_Tracker = []
            Indices,AllIndices,Mutated_Evolutionary_Complexity[p,:],Cell_Collection, Cell_Type_Tracker,KLDistance_temp,BitDistance_temp = Scoring_Indexes(Amount_Small_Perturbations,Final_Epigenetic_Factors[p,:,:,:],Cell_Type_Tracker)

        end
        Mutated_Mean[Counter_EMD_Index] = mean(Mutated_Evolutionary_Complexity/2^Splits) 
        Mutated_Std[Counter_EMD_Index] = std(Mutated_Evolutionary_Complexity/2^Splits) 

        
    end
    PlotterdePlot = plot(Load_Generations[1:Counter_EMD_Index],Initial_Mean[1:Counter_EMD_Index],yerr =Initial_Std[1:Counter_EMD_Index],colour="blue",markerstrokecolor = :auto,label="Original")
    PlotterdePlot = plot!(Load_Generations[1:Counter_EMD_Index],Mutated_Mean[1:Counter_EMD_Index],yerr =Mutated_Std[1:Counter_EMD_Index],colour="red",markerstrokecolor = :auto,label="Mutated")
    display(PlotterdePlot)
end